# Librerías 

In [ ]:
#manipulación de datos
import pandas as pd
pd.set_option('display.max_columns',500)
pd.set_option('display.max_colwidth',-1)

#procesamiento de lenguaje natural
from nltk.corpus import stopwords as sw

#visualización
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
%matplotlib inline

#machine learning
from sklearn.mixture import GaussianMixture
from sklearn.manifold import MDS
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

# Datos

In [ ]:
df = pd.read_csv('pictoline_tweets.csv')
df = df[['text','favorite_count','retweet_count']]
df = df [(df.favorite_count>=31)&(df.favorite_count<=5566)]
df.reset_index(drop=True,inplace=True)
df.head()

# Inspección Visual

In [ ]:
X = df[['favorite_count','retweet_count']]

In [ ]:
scaler = MinMaxScaler()
Xs = pd.DataFrame(scaler.fit_transform(X),columns=['v_fc','V_rc'])
mds = MDS()
Xmds = mds.fit_transform(Xs)

In [ ]:
Xmds = pd.DataFrame(Xmds,columns=['dim1','dim2'])

In [ ]:
sns.lmplot(data=Xmds.sample(frac=0.1),x='dim1',y='dim2',fit_reg=False,size=7)

#  Modelación no supervisada

In [ ]:
gmm = GaussianMixture(n_components=3)
gmm.fit(Xs)
cluster = pd.Series(gmm.predict(Xs))

In [ ]:
pastel = cluster.value_counts()/len(cluster)

In [ ]:
plt.pie(pastel,labels=pastel.index)

In [ ]:
Xmds['cluster'] = X['cluster'] = df['cluster'] = cluster

In [ ]:
sns.lmplot(data=Xmds.sample(frac=0.1),x='dim1',y='dim2',fit_reg=False,size=7,hue='cluster')

** Perfil **

In [ ]:
pd.concat([X.groupby('cluster').mean().reset_index(drop=True),(pd.DataFrame(X.mean()).transpose()).drop('cluster',axis=1)],
          ignore_index=True)

# Tratamiento de texto

In [ ]:
Xt,Xv,yt,yv = train_test_split(df['text'],df['cluster'],train_size=0.7)

In [ ]:
Xt.head()

In [ ]:
for c in range(0,3):
    texto = Xt[cluster==c].to_string(index=False)
    texto=texto.split()
    stopwords=sw.words('spanish') 
    texto = [pal for pal in texto if pal not in stopwords] 
    texto = [pal for pal in texto if not pal.startswith('http')]
    texto = [pal for pal in texto if len(pal)>1]
    texto = " ".join(texto)

    wordcloud = WordCloud().generate(texto)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

In [ ]:
cvect=CountVectorizer(encoding='utf8',
                      stop_words=stopwords,
                      strip_accents='unicode',ngram_range=(1,2))

In [ ]:
cvect.fit(Xt)
dtmt=cvect.transform(Xt)
dtmv=cvect.transform(Xv)

In [ ]:
mnb=MultinomialNB()
mnb.fit(dtmt,yt)
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(dtmt,yt)
mlp=MLPClassifier(hidden_layer_sizes=(20,20))
mlp.fit(dtmt,yt)